<a href="https://colab.research.google.com/github/remesis/BeatsAI/blob/main/new3Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import os
import pandas as pd
import numpy as np
import json
import math
import librosa

In [ ]:
model_name = 'cnn_7738_BeatsAI.keras'
path = F"/content/drive/MyDrive/SavedModels/{model_name}"

In [ ]:
dataDir = "/content/drive/MyDrive/Data/genres_original"
jsonPath = "/content/drive/MyDrive/Data/data.json"
csvPath = "/content/drive/MyDrive/Data/features_30_sec.csv"

In [ ]:
with open(jsonPath, "r") as fp:
  data = json.load(fp)

X = np.array(data["mfcc"])
y = np.array(data["labels"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
modelLSTM = keras.Sequential()


modelLSTM.add(keras.layers.LSTM(64,input_shape = (X_train.shape[1], X_train.shape[2]), return_sequences=True))
modelLSTM.add(keras.layers.LSTM(128, return_sequences=True,kernel_regularizer=keras.regularizers.l2(0.01)))
modelLSTM.add(keras.layers.LSTM(64, return_sequences=True,kernel_regularizer=keras.regularizers.l2(0.01)))
modelLSTM.add(keras.layers.LSTM(64))


modelLSTM.add(keras.layers.Dense(128, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(64, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(32, activation='relu'))
modelLSTM.add(keras.layers.Dropout(0.3))

modelLSTM.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:
modelCNN = keras.Sequential()

modelCNN.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
modelCNN.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())

modelCNN.add(keras.layers.Conv2D(128, (3, 3), activation='relu'))
modelCNN.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())


modelCNN.add(keras.layers.Conv2D(64, (2, 2), activation='relu'))
modelCNN.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
modelCNN.add(keras.layers.BatchNormalization())

modelCNN.add(keras.layers.Flatten())

modelCNN.add(keras.layers.Dense(64, activation='relu'))
modelCNN.add(keras.layers.Dropout(0.2))

modelCNN.add(keras.layers.Dense(10, activation='softmax'))

In [ ]:

modelNN = keras.Sequential([
    keras.layers.Flatten(input_shape=(X.shape[1], X.shape[2])),
    keras.layers.Dense(2048, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(10, activation='softmax')
])

In [ ]:
modelLSTM.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 64)           19968     
                                                                 
 lstm_1 (LSTM)               (None, 130, 128)          98816     
                                                                 
 lstm_2 (LSTM)               (None, 130, 64)           49408     
                                                                 
 lstm_3 (LSTM)               (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8

In [ ]:
modelCNN.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 11, 128)      1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 6, 128)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 64, 6, 128)        512       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 62, 4, 128)        147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 31, 2, 128)        0         
 g2D)                                                            
                                                      

In [ ]:
modelNN.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 1690)              0         
                                                                 
 dense_6 (Dense)             (None, 2048)              3463168   
                                                                 
 dropout_4 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 1024)              2098176   
                                                                 
 dropout_5 (Dropout)         (None, 1024)              0         
                                                                 
 dense_8 (Dense)             (None, 512)               524800    
                                                                 
 dropout_6 (Dropout)         (None, 512)              

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0000000000001)
modelNN.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

historyNN = modelNN.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=48, epochs=100)


Epoch 1/100
138/138 [==============================] - 8s 9ms/step - loss: 43.9077 - accuracy: 0.1063 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 2/100
138/138 [==============================] - 1s 9ms/step - loss: 43.9645 - accuracy: 0.1043 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 3/100
138/138 [==============================] - 1s 8ms/step - loss: 43.6994 - accuracy: 0.1104 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 4/100
138/138 [==============================] - 1s 9ms/step - loss: 44.1375 - accuracy: 0.1040 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 5/100
138/138 [==============================] - 1s 7ms/step - loss: 44.0199 - accuracy: 0.1052 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 6/100
138/138 [==============================] - 1s 7ms/step - loss: 44.3706 - accuracy: 0.1057 - val_loss: 30.9115 - val_accuracy: 0.1243
Epoch 7/100
138/138 [==============================] - 1s 7ms/step - loss: 44.2455 - accuracy: 0.1069 - val_loss: 30.9115 - val_ac

In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)

modelLSTM.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
historyLSTM = modelLSTM.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)


In [ ]:
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
modelCNN.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

historyCNN = modelCNN.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=32, epochs=100)


In [ ]:
modelNN.evaluate(X_test,y_test,verbose=0)


[30.911455154418945, 0.12433768808841705]

In [ ]:
modelCNN.evaluate(X_test,y_test,verbose=0)


[0.7530624270439148, 0.8265630602836609]

In [ ]:
modelLSTM.evaluate(X_test,y_test,verbose=0)


[1.4805271625518799, 0.714235246181488]

In [ ]:
pip install --upgrade scikit-learn


In [ ]:
from sklearn.preprocessing import minmax_scale, MinMaxScaler, LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt


In [ ]:

y_pred_test = modelCNN.predict(X_test)
print(classification_report(y_test, y_pred_test, digits=3))
print('Test Accuracy = {:.4f}'.format(accuracy_score(y_test, y_pred_test)))
fig, ax = plt.subplots(figsize=(6, 6))
confusion_matrix(modelCNN, X_test, y_test, display_labels=["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"], cmap=plt.cm.Blues, xticks_rotation=90, ax=ax)
plt.show()

89/89 [==============================] - 0s 2ms/step


ValueError: ignored